In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import torch

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

import utils.torchUtils as gnn

plt.style.use('science')
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams['font.size'] =  15

In [2]:
tree = Tree(fc.passthrough.TESTING_MX_700_MY_400)

In [3]:
ak.argmax(tree.n_jet)

1662

In [4]:
testing = gnn.Dataset('data/signal-testing',transform=gnn.to_uptri_graph)
len(testing)

5619

In [7]:
import torch.nn.functional as F
from torch_geometric.nn import Linear, GCNConv
from utils.torchUtils.layers import EdgeConv, EdgeConvONNX

class GCN(torch.nn.Module):
    def __init__(self,nn1_out=32,nn2_out=64,for_onnx=False):
        super().__init__()
        
        EdgeConvLayer = EdgeConv
        if for_onnx: EdgeConvLayer = EdgeConvONNX
        
        nn1 = torch.nn.Sequential(
            Linear(2*testing.num_node_features +
                   testing.num_edge_features, nn1_out),
            torch.nn.ELU()
        )

        self.conv1 = EdgeConvLayer(nn1, edge_aggr=None, return_with_edges=True)

        nn2 = torch.nn.Sequential(
            Linear(5*nn1_out, nn2_out),
            torch.nn.ELU()
        )

        self.conv2 = EdgeConvLayer(nn2, edge_aggr=None, return_with_edges=True)

        self.edge_seq = torch.nn.Sequential(
            Linear(3*nn2_out, 2),
        )

        self.node_seq = torch.nn.Sequential(
            Linear(nn2_out, 2),
        )

    def forward(self, x, edge_index, edge_x):
        # x, edge_index = data.x, data.edge_index
        
        
        x, edge_x = self.conv1(x, edge_index, edge_x)
        x, edge_x = self.conv2(x, edge_index, edge_x)
        x, edge_x = self.node_seq(x), self.edge_seq(edge_x)

        return F.log_softmax(x, dim=1), F.log_softmax(edge_x, dim=1)

In [15]:
model = gnn.GCN.load_from_checkpoint('models/graph_classifier/lightning_logs/version_6/checkpoints/epoch=19-step=16119.ckpt',dataset=testing)
model_for_onnx = gnn.GCN.load_from_checkpoint('models/graph_classifier/lightning_logs/version_6/checkpoints/epoch=19-step=16119.ckpt',dataset=testing,for_onnx=True)


In [16]:
def get_inputs(graph,pad_nodes=None):
    node_x = graph.x 
    edge_index = graph.edge_index 
    edge_x = graph.edge_attr 
    
    if pad_nodes: 
        node_x = F.pad(node_x,(0,0,0,pad_nodes-graph.num_nodes))
    return (node_x,edge_index,edge_x)

In [20]:
def compare_outputs(outputs_1,outputs_2):
    with torch.no_grad():
        for out0,out1 in zip(outputs_1,outputs_2):
            print(out0.shape,out1.shape)
            print( ((out0-out1)**2).sum() )

In [21]:
input_values = get_inputs(testing[1662])
input_names = ['node_x','edge_index','edge_x']
output_names = ['node_y','edge_y']
list(map(lambda t:t.shape,input_values))


[torch.Size([14, 5]), torch.Size([2, 91]), torch.Size([91, 1])]

In [23]:
org_output = model(input_values)
new_output = model_for_onnx(input_values)

compare_outputs(org_output,new_output)

torch.Size([14, 2]) torch.Size([14, 2])
tensor(12.0052)
torch.Size([91, 2]) torch.Size([91, 2])
tensor(465.2822)


In [25]:
org_output[0],new_output[0]

(tensor([[-3.3215, -0.0368],
         [-1.0296, -0.4418],
         [-1.1045, -0.4026],
         [-3.4835, -0.0312],
         [-1.3002, -0.3181],
         [-2.3133, -0.1042],
         [-0.0415, -3.2018],
         [-0.0486, -3.0482],
         [-0.0059, -5.1285],
         [-0.0280, -3.5891],
         [-0.0330, -3.4279],
         [-0.0704, -2.6886],
         [-0.2757, -1.4231],
         [-0.0491, -3.0383]], grad_fn=<LogSoftmaxBackward>),
 tensor([[-3.3215, -0.0368],
         [-1.6012, -0.2252],
         [-1.5112, -0.2493],
         [-1.9930, -0.1465],
         [-0.3960, -1.1178],
         [-1.0657, -0.4223],
         [-0.1127, -2.2390],
         [-0.2344, -1.5656],
         [-0.0262, -3.6557],
         [-0.0146, -4.2336],
         [-0.0494, -3.0323],
         [-0.1102, -2.2601],
         [-0.2856, -1.3924],
         [-0.0496, -3.0276]], grad_fn=<LogSoftmaxBackward>))

In [70]:
import torch.onnx

torch.onnx.export(model,
                  input_values,
                  "gnn-model.onnx",
                  input_names=input_names,
                  output_names=output_names,
                  opset_version=12,
                  dynamic_axes={
                      'node_x': {0: 'n_nodes'},
                      'edge_index': {1: 'n_edges'},
                      'edge_x': {0: 'n_edges'},
                      'node_y': {0: 'n_nodes'},
                      'edge_y': {0: 'n_edges'}}
                  )


In [71]:
import onnx
onnx_model = onnx.load('gnn-model.onnx')
onnx.checker.check_model(onnx_model)


In [72]:
import onnxruntime as ort

ort_sess = ort.InferenceSession('gnn-model.onnx')

In [80]:
g = testing[0]
print(g)
input_values = get_inputs(g,pad_nodes=20)
output_values = model(*input_values)
onnx_output = ort_sess.run(None,{name:value.numpy() for name,value in zip(input_names,input_values)})


Data(x=[7, 5], edge_index=[2, 21], edge_attr=[21, 1], y=[7], edge_y=[21])
torch.Size([20, 2]) (20, 2)
tensor(0.)
torch.Size([21, 2]) (21, 2)
tensor(2.4869e-14)


In [83]:
np.exp(onnx_output[0])

array([[0.51562   , 0.48437998],
       [0.5112753 , 0.4887247 ],
       [0.50532067, 0.4946793 ],
       [0.5060546 , 0.49394533],
       [0.5042418 , 0.49575827],
       [0.5060317 , 0.4939682 ],
       [0.4981752 , 0.5018248 ],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998],
       [0.51562   , 0.48437998]], dtype=float32)